# Import Libraries

In [1]:
# Pandas & Numpy 
import pandas as pd
import numpy as np

# Tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs

tf.compat.v1.enable_eager_execution()

# Types
from typing import Dict, Text

# System
import os 
import pprint
import tempfile

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-07-06 11:26:46.045505: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading & Reading Data

In [2]:
# Read original Data to collect all columns
original_df  = pd.read_csv('../../00_Data/Rec_Sys_precleaned.csv', index_col=0)

In [3]:
# Convert CustomerID to String
original_df['CustomerID']  = original_df['CustomerID'].astype(str)

# Selection of Features

In [4]:
# Select important Features
interactions_dict = original_df.groupby(['CustomerID','StockCode']).agg({'CustomerID': 'count'})
interactions_dict.columns = ['purchased']
interactions_dict.reset_index(inplace=True)
# Set all purchases to 1 
interactions_dict['purchased'] = 1 

# Transform Table into a Dictionary, which will be fed into tensor slices
interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()} 
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

# We do a similar step for items, where this is the reference table for items to be recommended 
items_dict = original_df[['StockCode']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)


## map the features in interactions and items to an identifier that we will use throught the embedding layers
## do it for all the items in interaction and item table
## you may often get itemtype error, so that is why here i am casting the quantity type as float to ensure consistency
interactions = interactions.map(lambda x: {
    'user_id' : x['CustomerID'], 
    'product_id' : x['StockCode'], 
    'purchased' : float(x['purchased']),

})

items = items.map(lambda x: x['StockCode'])

# Train Test Split

In [5]:
unique_item_titles = np.unique(np.concatenate(list(items.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["user_id"]))))

2023-07-06 11:26:50.800114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2349]
	 [[{{node Placeholder/_0}}]]
2023-07-06 11:26:50.938125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_2}}]]


In [6]:
### get unique item and user id's as a lookup table
unique_item_titles = np.unique(np.concatenate(list(items.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["user_id"]))))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = interactions.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60_000)
test = shuffled.skip(60_000).take(20_000)

2023-07-06 11:26:53.578399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2349]
	 [[{{node Placeholder/_0}}]]
2023-07-06 11:26:53.602413: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_2}}]]


# Define Model Classes

In [7]:
class RetailModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        
        ### Candidate model (item)
        ### This is Keras preprocessing layers to first convert user ids to integers, 
        ### and then convert those to user embeddings via an Embedding layer. 
        ### We use the list of unique user ids we computed earlier as a vocabulary:
        item_model = tf.keras.Sequential([
                                        tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_item_titles, mask_token=None),
                                        tf.keras.layers.Embedding(len(unique_item_titles) + 1, embedding_dimension)
                                        ])
        ### we pass the embedding layer into item model
        self.item_model: tf.keras.Model = item_model
            
        ### Query model (users)    
        user_model = tf.keras.Sequential([
                                        tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_user_ids, mask_token=None),
                                        # We add an additional embedding to account for unknown tokens.
                                        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                        ])
        self.user_model: tf.keras.Model = user_model
        
        ### for retrieval model. we take top-k accuracy as metrics
        metrics = tfrs.metrics.FactorizedTopK(
                                            candidates=items.batch(128).map(item_model))
        
        # define the task, which is retrieval                                    )    
        task = tfrs.tasks.Retrieval(
                                    metrics=metrics
                                    )
       
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.item_model(features["product_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

# Fitting & Evaluation

In [8]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 32

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_item_titles, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_item_titles) + 1, embedding_dimension)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                ])

model = RetailModel(user_model, item_model)

# a smaller learning rate may make the model move slower and prone to overfitting, so we stick to 0.1
# other optimizers, such as SGD and Adam, are listed here https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

## fit the model with ten epochs
model_hist = model.fit(cached_train, epochs=10)

#evaluate the model
model.evaluate(cached_test, return_dict=True)

Epoch 1/10


2023-07-06 11:26:56.249961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [172701]
	 [[{{node Placeholder/_0}}]]
2023-07-06 11:26:56.250338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]
2023-07-06 11:26:59.428523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{nod

1/8 [==>...........................] - ETA: 28s - factorized_top_k/top_1_categorical_accuracy: 4.8828e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0034 - factorized_top_k/top_50_categorical_accuracy: 0.0204 - factorized_top_k/top_100_categorical_accuracy: 0.0399 - loss: 73817.5938 - regularization_loss: 0.0000e+00 - total_loss: 73817.5938

2023-07-06 11:27:00.340598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]]


2/8 [======>.......................] - ETA: 4s - factorized_top_k/top_1_categorical_accuracy: 3.6621e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0250 - factorized_top_k/top_100_categorical_accuracy: 0.0485 - loss: 73818.4414 - regularization_loss: 0.0000e+00 - total_loss: 73818.4414 

...

2023-07-06 11:27:57.798473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2349]
	 [[{{node Placeholder/_0}}]]


4/5 [=======================>......] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 4.2725e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0035 - factorized_top_k/top_10_categorical_accuracy: 0.0109 - factorized_top_k/top_50_categorical_accuracy: 0.0607 - factorized_top_k/top_100_categorical_accuracy: 0.1119 - loss: 35835.7969 - regularization_loss: 0.0000e+00 - total_loss: 35835.7969

2023-07-06 11:27:58.052970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]]


5/5 [==============================] - 4s 298ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0112 - factorized_top_k/top_50_categorical_accuracy: 0.0603 - factorized_top_k/top_100_categorical_accuracy: 0.1116 - loss: 34283.5169 - regularization_loss: 0.0000e+00 - total_loss: 34283.5169


{'factorized_top_k/top_1_categorical_accuracy': 0.00039999998989515007,
 'factorized_top_k/top_5_categorical_accuracy': 0.003700000001117587,
 'factorized_top_k/top_10_categorical_accuracy': 0.01119999960064888,
 'factorized_top_k/top_50_categorical_accuracy': 0.06030000001192093,
 'factorized_top_k/top_100_categorical_accuracy': 0.11159999668598175,
 'loss': 31178.95703125,
 'regularization_loss': 0,
 'total_loss': 31178.95703125}

# Predictions

In [203]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=10)

index.index_from_dataset(items.batch(100).map(lambda title: (title, model.item_model(title))))

2023-07-06 12:49:11.289770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]]


In [215]:
index(tf.constant(["15262"]))[1][0][1].numpy().decode()

'22459'

# Create Top K List per User

In [204]:
# First we select all users in a list 
all_users = original_df['CustomerID'].unique().tolist()

In [205]:
# Create Data Frame 
# First Set up empty lists 
c_name = []
i_name = []

# Iterate through all users 
for u in all_users:
    for i in range(10):
        ranked_item = index(np.array([u]))[1][0][i].numpy().decode()
        c_name.append(u)
        i_name.append(ranked_item)


# Create Dataframe         
df_predictions = pd.DataFrame({
    "CustomerID": c_name,
    "StockCode" : i_name
})

# Add Predictions Column
df_predictions['predictions'] = 1.0

In [206]:
# Create Test Dataset
test_list = list(test.as_numpy_iterator())
df_test = pd.DataFrame(test_list, columns=['user_id', 'product_id', 'purchased'])
df_test['user_id'] = df_test['user_id'].str.decode('utf-8')
df_test['product_id'] = df_test['product_id'].str.decode('utf-8')
df_test = df_test.rename(columns={'user_id':'CustomerID', 'product_id':'StockCode'})


In [207]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
prediction_items = set(df_predictions['CustomerID'].unique())
test_items = set(df_test['CustomerID'].unique())

# Find the intersection of the sets from step 1
common_items = prediction_items.intersection(test_items)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
df_predictions = df_predictions[df_predictions['CustomerID'].isin(common_items)]
df_test = df_test[df_test['CustomerID'].isin(common_items)]

In [208]:
# Send to csv for further assessment
df_predictions.to_csv('pred.csv',index=False)
df_test.to_csv('test.csv', index=False)